# Preprocessing

In [1]:
# # You may need to run those in your enviroment terminal.

%pip install -r requirements.txt

  Using cached numpy-1.24.3.tar.gz (10.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  Getting requirements to build wheel did not run successfully.
  exit code: 1
  
  [33 lines of output]
  Traceback (most recent call last):
    File "E:\anaconda\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
      main()
    File "E:\anaconda\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
      json_out['return_val'] = hook(**hook_input['kwargs'])
                               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    File "E:\anaconda\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 112, in get_requires_for_build_wheel
      backend = _build_backend()
                ^^^^^^^^^^^^^^^^
    File "E:\anaconda\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 77, in _build_backend
      obj = import_module(mod_path)
            ^^^^^^^^^^^^^^^^^^^^^^^
    File "E:\anaconda\Lib\importlib\__init__.py",

In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import spacy
import nltk
from nltk.corpus import wordnet
import random
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datasets import Dataset
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score
import torch
import joblib


nltk.download('wordnet')
nltk.download('omw-1.4')

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

Creating data frame of the data and assigning them labels

In [ ]:
texts = []
labels = []

base_path = Path('review_polarity/txt_sentoken')
pos_path = base_path / 'pos'
neg_path = base_path / 'neg'

# Assign label 1
if pos_path.exists():
    for file in pos_path.glob('*.txt'):
        with open(file, 'r', encoding = 'utf-8') as f:
            texts.append(f.read())
            labels.append(1)

# Assign label 0
if neg_path.exists():
    for file in neg_path.glob('*.txt'):
        with open(file, 'r', encoding = 'utf-8') as f:
            texts.append(f.read())
            labels.append(0)

df = pd.DataFrame({
    'label': labels,
    'text': texts
})

df.tail()

In [ ]:
df[df.duplicated()]

No duplicates were found

In [ ]:
def spacy_pos_to_wordnet_pos(spacy_pos):
    if spacy_pos.startswith('N'):
        return wordnet.NOUN
    elif spacy_pos.startswith('V'):
        return wordnet.VERB
    elif spacy_pos.startswith('J'):
        return wordnet.ADJ
    elif spacy_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default fallback

In [ ]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    synonyms.discard(word)      # Remove the original word to avoid replacement with itself
    return list(synonyms)

In [ ]:
def augment_text_with_synonyms(tokens, pos_tags, synonym_probability = 0.2):
    augmented_tokens = []

    for token, pos_tag in zip(tokens, pos_tags):
        if random.random() < synonym_probability:
            if pos_tag in ['n', 'v', 'a']:   #nouns adjectives and verbs
                synonyms = get_synonyms(token)
                if synonyms:
                    new_word = random.choice(synonyms)
                    augmented_tokens.append(new_word)
                    continue
        augmented_tokens.append(token)         # Add the original word if no augmentation is done

    return augmented_tokens

In [ ]:
def apply_augmentation_to_dataset(word_tokens, pos_tags, texts, labels, sentence_tokens, synonym_probability = 0.2):
    all_word_tokens = word_tokens.copy()
    all_texts = texts.copy()  # Original text is preserved
    all_labels = labels.copy()
    all_sentence_tokens = sentence_tokens.copy()
    all_pos_tags = pos_tags.copy()

    for tokens, pos, label, sentence, text in zip(word_tokens, pos_tags, labels, sentence_tokens, texts):
        augmented_tokens = augment_text_with_synonyms(tokens, pos, synonym_probability)

        all_word_tokens.append(augmented_tokens)
        all_texts.append(text)  # Keep the original text
        all_labels.append(label)
        all_pos_tags.append(pos)
        all_sentence_tokens.append(sentence)  # Sentence tokens are not augmented

    return all_word_tokens, all_pos_tags, all_texts, all_labels, all_sentence_tokens

In [ ]:
def apply_lemmatization(tokens):
    return [token.lemma_ for token in nlp(' '.join(tokens))]

In [ ]:
def apply_stemming(text):
    words = text.split()
    return ' '.join([PorterStemmer().stem(word) for word in words])

In [ ]:
def visualize_tf_idf_heatmap(tfidf_matrix, feature_names, n_top_features = 20, n_top_docs = 10):
    # Get the top features by summing TF-IDF scores across documents
    tfidf_array = tfidf_matrix.toarray()
    feature_importance = np.sum(tfidf_array, axis = 0)
    top_feature_indices = np.argsort(feature_importance)[-n_top_features:][::-1]
    top_features = [feature_names[i] for i in top_feature_indices]

    # Get the top documents by summing TF-IDF scores across features
    doc_importance = np.sum(tfidf_array, axis = 1)
    top_doc_indices = np.argsort(doc_importance)[-n_top_docs:][::-1]

    # Extract the submatrix for visualization
    sub_matrix = tfidf_array[np.ix_(top_doc_indices, top_feature_indices)]

    plt.figure(figsize = (12, 8))
    sns.heatmap(
        sub_matrix,
        annot = True,          # Show values in cells
        fmt = '.3f',           # Format with 3 decimal places
        cmap = 'YlGnBu',       # Better colormap
        xticklabels = top_features,
        yticklabels = range(n_top_docs)
    )
    plt.title(f'TF-IDF Heatmap (Top {n_top_features} Features, First {n_top_docs} Documents)')
    plt.xlabel('Words')
    plt.ylabel('Documents')
    plt.xticks(rotation = 45, ha = 'right')
    plt.tight_layout()
    plt.savefig("Visualization\\tfidf_heatmap.png")
    plt.close()

    # Bar chart of top features across the corpus
    plt.figure(figsize = (12, 6))
    top_features_scores = [feature_importance[i] for i in top_feature_indices]
    plt.bar(top_features, top_features_scores)
    plt.title('Top TF-IDF Features Across All Documents')
    plt.xlabel('Features')
    plt.ylabel('Sum of TF-IDF Scores')
    plt.xticks(rotation = 45, ha = 'right')
    plt.tight_layout()
    plt.savefig("Visualization\\tfidf_top_features.png")
    plt.close()

In [ ]:
texts = df['text'].tolist()
labels = df['label'].tolist()
word_tokens = []
sentence_tokens = []
pos_tags = []

for text in texts:
    doc = nlp(text)
    sentence_tokens.append([sent.text.strip() for sent in doc.sents])
    tokens = [token for token in doc if not token.is_punct and not token.is_stop and not token.is_space]
    word_tokens.append([token.text.lower() for token in tokens])
    pos_tags.append([spacy_pos_to_wordnet_pos(token.tag_) for token in tokens])

lemmatized_tokens = [apply_lemmatization(t) for t in word_tokens]

word_tokens, pos_tags, texts, labels, sentence_tokens = apply_augmentation_to_dataset(
    lemmatized_tokens, pos_tags, texts, labels, sentence_tokens
)

joined_texts = [' '.join(tokens) for tokens in word_tokens]

augmented_df = pd.DataFrame({
    'label': labels,
    'text': joined_texts,
    'word_tokens': word_tokens
})

augmented_df = augmented_df.sample(frac=1, random_state=42).reset_index(drop=True)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    augmented_df['text'], augmented_df['label'], test_size=0.2, random_state=42
)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
vectorizer.fit(train_texts)
xtrain_tfidf = vectorizer.transform(train_texts)
xtest_tfidf = vectorizer.transform(test_texts)

# --- Important: Do NOT fit the vectorizer again before transforming test data! ---
tfidf_matrix = vectorizer.transform(joined_texts) 
feature_names = vectorizer.get_feature_names_out()

print(f"\nTF-IDF Matrix Shape (Train): {xtrain_tfidf.shape}")
print(f"TF-IDF Matrix Shape (Test): {xtest_tfidf.shape}")
print(f"TF-IDF Matrix Shape (Full): {tfidf_matrix.shape}") 
print(f"Number of features: {len(feature_names)}")

visualize_tf_idf_heatmap(tfidf_matrix, feature_names)

filename = "Models\\Vectorizer.joblib"
joblib.dump(vectorizer, filename)

In [ ]:
print("augmented_df.head():")
display(augmented_df.head())

print("\ndf.head():")
display(df.head())

In [ ]:
print("augmented_df.tail():")
display(augmented_df.tail())

print("\ndf.tail():")
display(df.tail())

In [ ]:
print("\ndf info:")
display(df.info())

print("\naugmented_df info:")
display(augmented_df.info())

In [ ]:
print("\ndf dimentions:")
display(df.shape)

print("\naugmented_df dimentions:")
display(augmented_df.shape)

# Visualizations

In [ ]:
tf = augmented_df["text"].apply(lambda x: pd.value_counts(x.split(" "))).sum(axis=0).reset_index()
tf.head()

In [ ]:
# fix the column names
tf.columns = ["words", "tf"]

In [ ]:
# to see the most frequent words
tf.sort_values("tf", ascending=False)

In [ ]:
tf[tf["tf"] > 3000].plot.bar(x="words", y="tf")
plt.show()

In [ ]:
from wordcloud import WordCloud
# check every row and join with these with a space
text = " ".join(i for i in augmented_df.text)
wordcloud = WordCloud().generate(text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# Modelling - Classifiers

####        •    ML -> Logistic Regression, Naive Bayes, SVM, Decision tree, and Random Forest.

In [ ]:
def train_model(classifier, feature_vector_train, train_labels, feature_vector_test, test_labels, is_neural_net=False):
    classifier.fit(feature_vector_train, train_labels)

    train_predictions = classifier.predict(feature_vector_train)
    test_predictions = classifier.predict(feature_vector_test)
    train_accuracy = metrics.accuracy_score(train_labels, train_predictions)
    test_accuracy = metrics.accuracy_score(test_labels, test_predictions)

    print(f"Train Accuracy: {train_accuracy * 100:.2f}%")
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

    return test_accuracy  # Return the test accuracy

#### 1. Logistic Regression

In [ ]:
model_l1 = linear_model.LogisticRegression(penalty='l1', C=0.7, solver='liblinear', random_state=42)
model_l1.fit(xtrain_tfidf, train_labels)

accuracy = train_model(model_l1, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename_l1 = r"Models\LR_L1.joblib"
joblib.dump(model_l1, filename_l1)

with open(r"Accuracies\LR_L1.txt", "w") as file:
    file.write(str(accuracy * 100))

In [ ]:
model_l2 = linear_model.LogisticRegression(penalty='l2', C=0.7, solver='liblinear', random_state=42)
model_l2.fit(xtrain_tfidf, train_labels)

accuracy = train_model(model_l2, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename_l2 = r"Models\LR_L2.joblib"
joblib.dump(model_l2, filename_l2)

with open(r"Accuracies\LR_L2.txt", "w") as file:
    file.write(str(accuracy * 100))

#### 2. Naive Bayes

In [ ]:
model = naive_bayes.MultinomialNB(alpha=0.7)
accuracy = train_model(model, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename = "Models\\NB.joblib"
joblib.dump(model, filename)

with open("Accuracies\\NB.txt", "w") as file:
    file.write(str(accuracy * 100))

#### 3. SVM

In [ ]:
model = svm.SVC(kernel='linear', C=0.3)
accuracy = train_model(model, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename = "Models\\SVM_Linear.joblib"
joblib.dump(model, filename)

with open("Accuracies\\SVM_Linear.txt", "w") as file:
    file.write(str(accuracy * 100))

In [ ]:
model = svm.SVC(kernel='rbf', C=0.3)
model.fit(xtrain_tfidf, train_labels)

accuracy = train_model(model, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

joblib.dump(model, r"Models\SVM_rbf.joblib")

with open(r"Accuracies\SVM_rbf.txt", "w") as file:
    file.write(str(accuracy * 100))

In [ ]:
model = svm.SVC(kernel='poly', C=0.3, degree=4)
accuracy = train_model(model, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename = "Models\\SVM_Poly.joblib"
joblib.dump(model, filename)

with open("Accuracies\\SVM_Poly.txt", "w") as file:
    file.write(str(accuracy * 100))

#### 4. Decision Tree

In [ ]:
model = DecisionTreeClassifier(max_depth=10, min_samples_split=5, min_samples_leaf=3, random_state=42)
accuracy = train_model(model, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename = "Models\\DT.joblib"
joblib.dump(model, filename)

with open("Accuracies\\DT.txt", "w") as file:
    file.write(str(accuracy * 100))

#### 5. Random Forest

In [ ]:
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    max_features='sqrt',
    random_state=42
)

accuracy = train_model(model, xtrain_tfidf, train_labels, xtest_tfidf, test_labels)

filename = "Models\\RF.joblib"
joblib.dump(model, filename)

with open("Accuracies\\RF.txt", "w") as file:
    file.write(str(accuracy * 100))

In [ ]:
from sklearn.metrics import confusion_matrix

def print_confusion_matrix(name, model, X_test, y_test):
    try:
        y_pred = model.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='coolwarm', cbar=False)
        plt.title(f'Confusion Matrix for {name}')
        plt.xlabel('Predicted Label (0=Negative, 1=Positive)')
        plt.ylabel('True Label (0=Negative, 1=Positive)')
        plt.tight_layout()

        sanitized_name = name.replace(' ', '_').replace('(', '').replace(')', '')
        plt.savefig(f'Visualization/confusion_matrix_{sanitized_name}.png')
        plt.show()
    except Exception as e:
        print(f"Error generating confusion matrix for {name}: {str(e)}")


In [ ]:
models = {
    'Logistic Regression (L1)': joblib.load('Models/LR_L1.joblib'),
    'Logistic Regression (L2)': joblib.load('Models/LR_L2.joblib'),
    'Naive Bayes': joblib.load('Models/NB.joblib'),
    'SVM (Linear)': joblib.load('Models/SVM_Linear.joblib'),
    'SVM (RBF)': joblib.load('Models/SVM_rbf.joblib'),
    'SVM (Poly)': joblib.load('Models/SVM_Poly.joblib'),
    'Decision Tree': joblib.load('Models/DT.joblib'),
    'Random Forest': joblib.load('Models/RF.joblib')
}

for name, model in models.items():
    print_confusion_matrix(name, model, xtest_tfidf, test_labels)

#### • DL -> BERT

In [ ]:
# Use same train/test split for BERT
bert_train_texts = train_texts.tolist()
bert_test_texts = test_texts.tolist()
bert_train_labels = train_labels.tolist()
bert_test_labels = test_labels.tolist()

bert_train_ds = Dataset.from_dict({"text": bert_train_texts, "label": bert_train_labels})
bert_test_ds = Dataset.from_dict({"text": bert_test_texts, "label": bert_test_labels})

In [ ]:
# Load the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the datasets
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=256)

bert_train_ds = bert_train_ds.map(tokenize_function, batched=True)
bert_test_ds = bert_test_ds.map(tokenize_function, batched=True)

# Set PyTorch format
bert_train_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])
bert_test_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
# Load pre-trained BERT for classification
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Define evaluation metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    return {"accuracy": accuracy_score(labels, predictions)}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=bert_train_ds,
    eval_dataset=bert_test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
# Evaluate BERT model on test set
metrics = trainer.evaluate()
print(f"BERT Classification Accuracy: {metrics['eval_accuracy']:.4f}")

In [ ]:
os.makedirs("Visualization", exist_ok=True)

vectorizer = joblib.load("Models/Vectorizer.joblib")

xtrain_tfidf = vectorizer.transform(train_texts)
xtest_tfidf = vectorizer.transform(test_texts)

print(f"Recomputed TF-IDF Matrix Shape (Train): {xtrain_tfidf.shape}")
print(f"Recomputed TF-IDF Matrix Shape (Test): {xtest_tfidf.shape}")

models = {
    'Logistic Regression (L1)': joblib.load('Models/LR_L1.joblib'),
    'Logistic Regression (L2)': joblib.load('Models/LR_L2.joblib'),
    'Naive Bayes': joblib.load('Models/NB.joblib'),
    'SVM (Linear)': joblib.load('Models/SVM_Linear.joblib'),
    'SVM (RBF)': joblib.load('Models/SVM_rbf.joblib'),
    'SVM (Poly)': joblib.load('Models/SVM_Poly.joblib'),
    'Decision Tree': joblib.load('Models/DT.joblib'),
    'Random Forest': joblib.load('Models/RF.joblib')
}

results = []
for model_name, sk_model in models.items():
    train_pred = sk_model.predict(xtrain_tfidf)
    test_pred = sk_model.predict(xtest_tfidf)
    train_accuracy = accuracy_score(train_labels, train_pred)
    test_accuracy = accuracy_score(test_labels, test_pred)
    results.append({
        'Model': model_name,
        'Train Accuracy': train_accuracy,
        'Test Accuracy': test_accuracy
    })

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='Train Accuracy', ascending=False).reset_index(drop=True)

print("Model Accuracy Comparison (Sorted by Train Accuracy)")
display(results_df)

plt.figure(figsize=(10, 6))
bar_width = 0.35
index = range(len(results_df))

plt.bar(index, results_df['Train Accuracy'], bar_width, label='Train Accuracy', color='skyblue')
plt.bar([i + bar_width for i in index], results_df['Test Accuracy'], bar_width, label='Test Accuracy', color='lightcoral')

plt.xlabel('Model')
plt.ylabel('Accuracy')
plt.title('Train and Test Accuracy Comparison Across Models')
plt.xticks([i + bar_width / 2 for i in index], results_df['Model'], rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.savefig('Visualization/model_accuracy_comparison.png')
plt.show()

In [ ]:
from sklearn.metrics import auc, roc_curve


plt.figure(figsize=(10, 6))
colors = ['blue', 'red', 'green', 'purple', 'orange', 'brown', 'pink', 'gray']

for (model_name, sk_model), color in zip(models.items(), colors):
    if hasattr(sk_model, 'predict_proba'):
        probs = sk_model.predict_proba(xtest_tfidf)[:, 1]
        fpr, tpr, _ = roc_curve(test_labels, probs)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})', color=color)

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Scikit-learn Models')
plt.legend(loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.savefig('Visualization/roc_curve_all_models.png')
plt.show()

In [ ]:
from sklearn.manifold import TSNE

model_files = {
    'LR_L1': 'Models/LR_L1.joblib',
    'LR_L2': 'Models/LR_L2.joblib',
    'Naive Bayes': 'Models/NB.joblib',
    'SVM_Linear': 'Models/SVM_Linear.joblib',
    'SVM_RBF': 'Models/SVM_rbf.joblib',
    'SVM_Poly': 'Models/SVM_Poly.joblib',
    'Decision Tree': 'Models/DT.joblib',
    'Random Forest': 'Models/RF.joblib'
}

tsne_tfidf = TSNE(n_components=2, random_state=42, perplexity=30, n_jobs=-1)
tsne_tfidf_results = tsne_tfidf.fit_transform(xtest_tfidf.toarray())

tsne_tfidf_df = pd.DataFrame({
    'TSNE1': tsne_tfidf_results[:, 0],
    'TSNE2': tsne_tfidf_results[:, 1],
    'True Label': test_labels
})

test_texts_reset = test_texts.reset_index(drop=True)

plt.figure(figsize=(10, 6))
sns.scatterplot(x='TSNE1', y='TSNE2', hue='True Label', palette='coolwarm', data=tsne_tfidf_df, alpha=0.7)
plt.title('t-SNE Visualization of TF-IDF Features (True Labels)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend(title='True Label (0=Negative, 1=Positive)', loc='best')
plt.grid(True)
plt.tight_layout()

np.random.seed(42)
for i in np.random.choice(len(test_texts_reset), 5, replace=False):
    plt.text(tsne_tfidf_df['TSNE1'].iloc[i], tsne_tfidf_df['TSNE2'].iloc[i], 
             test_texts_reset.iloc[i][:20], fontsize=8, ha='center')

plt.savefig('Visualization/tsne_tfidf_true.png')
plt.show()

for model_name, model_path in model_files.items():
    if os.path.exists(model_path):
        model = joblib.load(model_path)
        
        predictions = model.predict(xtest_tfidf)
        tsne_tfidf_df['Predicted Label'] = predictions

        plt.figure(figsize=(10, 6))
        sns.scatterplot(x='TSNE1', y='TSNE2', hue='Predicted Label', palette='coolwarm', data=tsne_tfidf_df, alpha=0.7)
        plt.title(f't-SNE Visualization of TF-IDF Features ({model_name} Predictions)')
        plt.xlabel('t-SNE Component 1')
        plt.ylabel('t-SNE Component 2')
        plt.legend(title='Predicted Label (0=Negative, 1=Positive)', loc='best')
        plt.grid(True)
        plt.tight_layout()

        np.random.seed(42)
        for i in np.random.choice(len(test_texts_reset), 5, replace=False):
            plt.text(tsne_tfidf_df['TSNE1'].iloc[i], tsne_tfidf_df['TSNE2'].iloc[i], 
                     test_texts_reset.iloc[i][:20], fontsize=8, ha='center')

        plt.savefig(f'Visualization/tsne_tfidf_{model_name.replace(" ", "_")}_predictions.png')
        plt.show()
    else:
        print(f"Model file {model_path} not found. Skipping {model_name}.")

In [ ]:
from sklearn.decomposition import PCA

pca_tfidf = PCA(n_components=2, random_state=42)
pca_tfidf_results = pca_tfidf.fit_transform(xtest_tfidf.toarray())

pca_tfidf_df = pd.DataFrame({
    'PC1': pca_tfidf_results[:, 0],
    'PC2': pca_tfidf_results[:, 1],
    'True Label': test_labels
})

test_texts_reset = test_texts.reset_index(drop=True)

plt.figure(figsize=(10, 6))
sns.scatterplot(x='PC1', y='PC2', hue='True Label', palette='coolwarm', data=pca_tfidf_df, alpha=0.7)
plt.title('PCA Visualization of TF-IDF Features (True Labels)')
plt.xlabel(f'PC1 ({pca_tfidf.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca_tfidf.explained_variance_ratio_[1]:.2%} variance)')
plt.legend(title='True Label (0=Negative, 1=Positive)', loc='best')
plt.grid(True)
plt.tight_layout()

np.random.seed(42)
for i in np.random.choice(len(test_texts_reset), 5, replace=False):
    plt.text(pca_tfidf_df['PC1'].iloc[i], pca_tfidf_df['PC2'].iloc[i], 
             test_texts_reset.iloc[i][:20], fontsize=8, ha='center')

plt.savefig('Visualization/pca_tfidf_true.png')
plt.show()

for model_name, model_path in model_files.items():
    if os.path.exists(model_path):
        model = joblib.load(model_path)
        
        predictions = model.predict(xtest_tfidf)
        pca_tfidf_df['Predicted Label'] = predictions

        plt.figure(figsize=(10, 6))
        sns.scatterplot(x='PC1', y='PC2', hue='Predicted Label', palette='coolwarm', data=pca_tfidf_df, alpha=0.7)
        plt.title(f'PCA Visualization of TF-IDF Features ({model_name} Predictions)')
        plt.xlabel(f'PC1 ({pca_tfidf.explained_variance_ratio_[0]:.2%} variance)')
        plt.ylabel(f'PC2 ({pca_tfidf.explained_variance_ratio_[1]:.2%} variance)')
        plt.legend(title='Predicted Label (0=Negative, 1=Positive)', loc='best')
        plt.grid(True)
        plt.tight_layout()

        np.random.seed(42)
        for i in np.random.choice(len(test_texts_reset), 5, replace=False):
            plt.text(pca_tfidf_df['PC1'].iloc[i], pca_tfidf_df['PC2'].iloc[i], 
                     test_texts_reset.iloc[i][:20], fontsize=8, ha='center')

        plt.savefig(f'Visualization/pca_tfidf_{model_name.replace(" ", "_")}_predictions.png')
        plt.show()
    else:
        print(f"Model file {model_path} not found. Skipping {model_name}.")